In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("dpschool2a.csv", delimiter=";")

In [3]:
df = df[['title', 'short_description', 'description', 'latitude', 'longitude', 'address', 'addr_no', 
         'zip_code', 'city', 'country', 'additional_fields', 'image', 'icons', 'modified']]

In [4]:
import json
df["additional_fields"] = df["additional_fields"].map(lambda x: json.loads(x))
df["image"] = df["image"].map(lambda x: json.loads(x))

In [5]:
df = df[df['latitude'].notna()]

In [6]:
from pandas import isnull
df["icons"] = df["icons"].map(lambda x: {} if isnull(x) else x)
df["icons"] = df["icons"].map(lambda x: x if isinstance(x, str) else str(x))
df["icons"] = df["icons"].map(lambda x: json.loads(x))

In [7]:
df["modified"] = pd.to_datetime(df["modified"], errors="coerce")

In [8]:
df['longitude'] = df['longitude'].map(lambda x: x.replace('.', "")[:6]).astype(float) / 10000
df['latitude'] = df['latitude'].map(lambda x: x.replace('.', "")[:6]).astype(float) / 10000

In [9]:
df["description"] = df["description"].map(lambda x: "" if isnull(x) else x)
df["short_description"] = df["short_description"].map(lambda x: "" if isnull(x) else x)
df["addr_no"] = df["addr_no"].map(lambda x: "" if isnull(x) else x)

In [10]:
df["address"] = df["address"] + " " + df["addr_no"]

In [11]:
df.drop(["addr_no"], axis=1, inplace=True)

In [12]:
df["position"] = None
for i in df.index:
    df.at[i, 'position'] = (df.loc[i]['latitude'], df.loc[i]['longitude'])

In [13]:
icons = {"Toilette für Alle": 10, "Rolli WC": 11, "Rolli WC -DIN fern": 20, "WC eng": 17, "kein Rolli WC vorhanden": 21,
        "Parken": 7, "Stufe": 1, "WC - unisex": 4, "WC getrennt": 6, "Rollstuhl": 5, "Lift groß": 12, "Lift klein": 13,
        "Kino-Untertitel": 15, "Induktionsschleife": 14, "low-vision": 18, "kognitiv beeinträchtigt": 19, "Umkleide": 9,
        "Dusche": 8, "nicht rollstuhlgerecht": 16, "Rampe": 2, "keine Steigung - eben": 26, "moderate Steigung / Gefälle": 22,
        "weitere moderate Steigungen / Gefälle": 23, "heftige Steigung / Gefälle": 25, "weitere heftige Steigung / Gefälle": 24}
new_dict = {v: k for k, v in icons.items()}
new_dict.update({40: "Euro Key", 41: "Ramp Incline", 42: "Door Width"})

In [14]:
df["icons"] = df["icons"].map(lambda x: list(map(int, list(x.values()))))

In [15]:
df[40] = df["additional_fields"].map(lambda x: x["wc_euro"] if "wc_euro" in x.keys() else "0")
df[40] = df[40].map(lambda x: int(x) if x in ["1", "0"] else 0)

In [16]:
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

df[41] = df["additional_fields"].map(lambda x: x["wc_ramp_incline"])
df[41] = df[41].map(lambda x: x.replace(",","."))
df[41] = df[41].map(lambda x: float(x) if isfloat(x) else 0)
#need to check further as some have extra text

In [17]:
df[42] = df["additional_fields"].map(lambda x: x["wc_door_width"])
df[42] = df[42].map(lambda x: int(x) if x != "" else 150)
df[42] = df[42].map(lambda x: x if x > 15 else x*10)

In [18]:
x = pd.get_dummies(df.icons.apply(pd.Series).stack()).sum(level=0)
x = x.rename(int, axis="columns")
col = list(x.columns)
for i in [40, 41, 42]:
    col.append(i)

In [19]:
df = df.rename(columns={40:"eurokey", 41: "ramp_steepness", 42: "door_width"})

In [20]:
# df.drop(["icons"], axis=1, inplace=True)
# df = pd.concat([df, x], axis=1)

In [21]:
# def check(series):
#     x = {}
#     for i in series.index:
#         if i in col:
#             x.update({i: series[i]})
#         else:
#             continue
#     return x

In [22]:
# df["options"] = df.apply(lambda x: check(x), axis=1)

In [23]:
# df.drop(col, axis=1, inplace=True)

In [24]:
# columns = pd.DataFrame.from_dict(new_dict, orient="index", columns=["option_name"])
# columns.index.name = "id"

In [25]:
# columns.to_csv("options.csv")

In [26]:
df["photo"] = df["image"].map(lambda x: x["image_small"])
df["plan"] = df["additional_fields"].map(lambda x: "images/wc-plan/" + x["wc_plan"] + ".jpg")
#df["seat_height"] = df["additional_fields"].map(lambda x: x["wc_seat"])

In [27]:
#df["seat_height"] = df["seat_height"].map(lambda x: x[:2] if x != "" else None)

In [28]:
df.head()

,title,short_description,description,latitude,longitude,address,zip_code,city,country,additional_fields,image,icons,modified,position,eurokey,ramp_steepness,door_width,photo,plan
0,WC U-Bahnhof Stiglmaierplatz,Die behindertengerechte Toilette ist geschloss...,,48.1475,11.5586,Johann-Fichte-Str. 12,80805,MÃ¼nchen,DE,"{'wc_shortname': 'bf-stiglm1WC', 'wc_plan': 'b...",{'image_small': 'images/wc-af/a-WC-verschwunde...,[21],2021-10-06 08:10:00,"(48.1475, 11.5586)",1,0.0,100,images/wc-af/a-WC-verschwunden.jpg,images/wc-plan/bf-stiglm1WC1.jpg
1,WC Bad Georgenschwaige Damen,Bedingt Ã¶ffentliche Toilette in einem Freibad.,<p>Das <strong>Damen-WC</strong> ist im Umklei...,48.1756,11.5717,Belgradstr. 195,80804,MÃ¼nchen,DE,"{'wc_shortname': 'ba-george1WC1', 'wc_plan': '...",{'image_small': 'images/wc-af/ba-george1WC1f1....,"[4, 11]",2021-09-08 09:21:00,"(48.1756, 11.5717)",0,0.0,85,images/wc-af/ba-george1WC1f1.jpg,images/wc-plan/ba-george1WC1.jpg
2,WC Bad Georgenschwaige Herren,Bedingt Ã¶ffentliche Toilette in einem Freibad.,<p>Das <strong>Herren-WC ist im Umkleidebau</s...,48.1756,11.5717,Belgradstr. 195,80804,MÃ¼nchen,DE,"{'wc_shortname': 'ba-george1WC2', 'wc_plan': '...",{'image_small': 'images/wc-af/ba-george1WC2f1....,"[6, 20]",2021-09-08 09:21:00,"(48.1756, 11.5717)",0,0.0,85,images/wc-af/ba-george1WC2f1.jpg,images/wc-plan/ba-george1WC2.jpg
3,WC Freibad Unterhaching,Bedingt Ã¶ffentliche Toilette mit Dusche in ei...,<p>Das rollstuhlgerechte WC des Unterhachinger...,48.0619,11.6208,Schrenkstr. 2,82008,Unterhaching,Deutschland,"{'wc_shortname': 'ba-unterh1WC', 'wc_plan': 'b...",{'image_small': 'images/wc-af/ba-unterh1WC1f1....,"[4, 8, 11]",2021-10-06 08:10:00,"(48.0619, 11.6208)",1,0.0,75,images/wc-af/ba-unterh1WC1f1.jpg,images/wc-plan/ba-unterh1WC.jpg
4,WC U-Bahnhof MÃ¼nchner Freiheit,"Ã–ffentliche Toilette im U-Bahnhof, zugÃ¤nglic...",<p>Das rollstuhlgerechte WC des U-Bahnhofs MÃ¼...,48.1622,11.5870,MÃ¼nchner Freiheit,80802,MÃ¼nchen,DE,"{'wc_shortname': 'bf-muefre1WC', 'wc_plan': 'b...",{'image_small': 'images/wc-af/bf-muefre1WC1f1....,"[4, 11, 12]",2021-10-06 08:09:00,"(48.1622, 11.587)",1,0.0,85,images/wc-af/bf-muefre1WC1f1.jpg,images/wc-plan/bf-muefre1WC1.jpg


In [ ]:
df[df['icons'].isin([[21]])].index
#df = df.drop(df.loc[df['icons'].isin([[21]])].index)

In [ ]:
df = df[["title", "short_description", "photo", "plan", "position", "address", "eurokey", "ramp_steepness", "door_width"]]

In [ ]:
df.head()

In [ ]:
errors = pd.read_excel("toilets_text errors.xlsx")

In [ ]:
errors.dropna(inplace=True)

In [ ]:
list(errors["Text error"])

In [ ]:
import re

errors = pd.Series(errors["correction"].values,index=errors["Text error"]).to_dict()

# use these three lines to do the replacement
rep = dict((re.escape(k), v) for k, v in errors.items()) 
#Python 3 renamed dict.iteritems to dict.items so use rep.items() for latest versions
pattern = re.compile("|".join(rep.keys()))

In [ ]:
df["short_description"] = df["short_description"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))
df["title"] = df["title"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))

In [ ]:
df.head()

In [ ]:
df.ramp_steepness.value_counts()

In [ ]:
df.info()

In [ ]:
df.to_json("toilets_v5.json")

In [1]:
import pandas as pd

In [6]:
df = pd.read_json("toilets_v6.json")

In [7]:
df.head()

,title,short_description,photo,plan,position,address,eurokey,ramp_steepness,door_width
1,WC Bad Georgenschwaige Damen,Bedingt öffentliche Toilette in einem Freibad.,get_image/images/wc-af/ba-george1WC1f1.jpg,get_layout/images/wc-plan/ba-george1WC1.jpg,"[48.1756, 11.5717]",Belgradstr. 195,0,0.0,85
2,WC Bad Georgenschwaige Herren,Bedingt öffentliche Toilette in einem Freibad.,get_image/images/wc-af/ba-george1WC2f1.jpg,get_layout/images/wc-plan/ba-george1WC2.jpg,"[48.1756, 11.5717]",Belgradstr. 195,0,0.0,85
3,WC Freibad Unterhaching,Bedingt öffentliche Toilette mit Dusche in ein...,get_image/images/wc-af/ba-unterh1WC1f1.jpg,get_layout/images/wc-plan/ba-unterh1WC.jpg,"[48.0619, 11.6208]",Schrenkstr. 2,1,0.0,75
4,WC U-Bahnhof Münchner Freiheit,"Öffentliche Toilette im U-Bahnhof, zugänglich ...",get_image/images/wc-af/bf-muefre1WC1f1.jpg,get_layout/images/wc-plan/bf-muefre1WC1.jpg,"[48.1622, 11.587]",MÃ¼nchner Freiheit,1,0.0,85
5,WC Dynasty Unterhaching,Halböffentliche Toilette in einem Lokal.,get_image/images/wc-af/dynast1WC1f1.jpg,get_layout/images/wc-plan/dynast1WC.jpg,"[48.0556, 11.6076]",Inselkammerstr. 3,0,0.0,90


In [8]:
df["id"] = df.index

In [9]:
df.head()

,title,short_description,photo,plan,position,address,eurokey,ramp_steepness,door_width,id
1,WC Bad Georgenschwaige Damen,Bedingt öffentliche Toilette in einem Freibad.,get_image/images/wc-af/ba-george1WC1f1.jpg,get_layout/images/wc-plan/ba-george1WC1.jpg,"[48.1756, 11.5717]",Belgradstr. 195,0,0.0,85,1
2,WC Bad Georgenschwaige Herren,Bedingt öffentliche Toilette in einem Freibad.,get_image/images/wc-af/ba-george1WC2f1.jpg,get_layout/images/wc-plan/ba-george1WC2.jpg,"[48.1756, 11.5717]",Belgradstr. 195,0,0.0,85,2
3,WC Freibad Unterhaching,Bedingt öffentliche Toilette mit Dusche in ein...,get_image/images/wc-af/ba-unterh1WC1f1.jpg,get_layout/images/wc-plan/ba-unterh1WC.jpg,"[48.0619, 11.6208]",Schrenkstr. 2,1,0.0,75,3
4,WC U-Bahnhof Münchner Freiheit,"Öffentliche Toilette im U-Bahnhof, zugänglich ...",get_image/images/wc-af/bf-muefre1WC1f1.jpg,get_layout/images/wc-plan/bf-muefre1WC1.jpg,"[48.1622, 11.587]",MÃ¼nchner Freiheit,1,0.0,85,4
5,WC Dynasty Unterhaching,Halböffentliche Toilette in einem Lokal.,get_image/images/wc-af/dynast1WC1f1.jpg,get_layout/images/wc-plan/dynast1WC.jpg,"[48.0556, 11.6076]",Inselkammerstr. 3,0,0.0,90,5


In [10]:
df.to_json("toilets_v7.json")